In [1]:
import cv2
import pandas as pd
import torch
from torch import nn
from torch.cuda import amp
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import timm
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda'

## Configuration

In [5]:
CFG = {
    'folds' : 5,
    'seed' : 872,
    'img_size' : 224,
    'model': 'efficientnet_b0',
    'epochs' : 10, 
    'train_bs': 10,
    'val_bs': 10,
    'lr': 1e-3,
    'amp': True,
}

## DATASET

In [6]:
class MelData(Dataset):
    
    def __init__(self, df: pd.DataFrame, train_augmentation = None, val_augmentation = None, train: bool = False, val: bool = False,):
        super().__init__()
        self.df = df
        self.train = train
        self.val = val
        self.train_augmentation = train_augmentation
        self.val_augmentation = val_augmentation
        self.path = 'data/train/train/'
        
    def _load_image(self, idx):
        path= self.path + self.df.iloc[idx]['image_name'] + '.jpg'
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img
    
    def _augment(self, img, fn):
        transformed = fn(image=img)
        transformed_image = transformed["image"]
        
        return transformed_image
    
    def __getitem__(self, idx):
        img = self._load_image(idx)
        
        if self.train:
            img = self._augment(img, self.train_augmentation)
            return img, self.df.iloc[idx]['target']
        if self.val:
            img = self._augment(img, self.val_augmentation)
            return img, self.df.iloc[idx]['target']
        
        return img
        
    def __len__(self, ):
        return len(self.df)
    

In [7]:
train_df = pd.read_csv('data/train_concat.csv')
train_df = train_df.iloc[:len(train_df)//2, :]

In [8]:
def view_img(image):
    cv2.namedWindow('Display', cv2.WINDOW_NORMAL)
    cv2.imshow('Display', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
transformation = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])

In [10]:
def calc_acc(z, y):
    
    out = np.where(z > .5, 1, 0)
    
    tp = out-y
    return (tp[tp == 0].shape[0])/y.shape[0]
    

In [11]:
def train(model, optim, criterion, scheduler,train_loader, val_loader, fold):
    
    if CFG['amp']:
        scaler = amp.GradScaler()
    
    for i in range(1, CFG['epochs'] + 1):
        
        model.train()
        epoch_loss = 0
        batch_acc = []
        best_val = 0
        
        
        
        for x, y in train_loader:
            x = x.to(device).type(torch.float32)
            y = y.to(device).type(torch.float32)
            optim.zero_grad()
            
            if CFG['amp']:
                
                with amp.autocast():
                    z = model(x)
                    loss = criterion(z.squeeze(-1), y)

                scaler.scale(loss).backward()
                scaler.step(optim)
                scaler.update()                
            else:
                
                z = model(x)
                loss = criterion(z.squeeze(-1), y)
                loss.backward()
                optim.step()
            
            epoch_loss += loss.item()
            batch_acc.append(calc_acc(torch.sigmoid(z.squeeze(-1)).detach().cpu().numpy(), y.detach().cpu().numpy()))
            

        acc = np.mean(batch_acc)
        
        print("loss is %.3f and accuracy is %.3f" %(epoch_loss, acc))
        
        val_auc = validate(model, criterion, val_loader)
        scheduler.step(val_auc)
        
        if val_auc >= best_val:
            best_val = val_auc
            save_model(model, 'test', fold) 
        if i == CFG['epochs']:
            save_model(model, 'last', fold)
            

In [12]:
def validate(model, criterion,data_loader):
    model.eval()
    
    with torch.no_grad():
        
        val_loss = 0
        y_true = []
        y_pred = []
        
        for x, y in data_loader:
            
            x = x.to(device).type(torch.float32)
            y = y.to(device).type(torch.float32)            
            
            z = model(x)
            loss = criterion(z.squeeze(-1), y)
            
            val_loss += loss.item()
            
            y_true.append(y.detach().cpu().numpy())
            y_pred.append(torch.sigmoid(z.squeeze(-1)).detach().cpu().numpy())
    
    y_true = np.stack(y_true)
    y_pred = np.stack(y_pred)
    val_auc = roc_auc_score(y_true, y_pred)
            
    print("loss is %.3f and auc is %.3f" %(val_loss, val_auc))
    
    return val_auc
    

In [13]:
def save_model(model, name, fold):
    path = f'models/{name}'
    if not os.path.exists(path):
        os.mkdir(path)
    model_name = CFG['model']
    torch.save(model.state_dict(), f'{path}/{model_name}_{fold}.pt') 

In [14]:
skf = StratifiedKFold(n_splits=5, random_state=999, shuffle=True)

In [15]:
folds = skf.split(X=np.zeros(len(train_df)), y=train_df.loc[: ,['target']])

In [ ]:

for fold, (train_idx, val_idx) in enumerate(folds,1):
    
    train_dataset = MelData(df=train_df.iloc[train_idx].reset_index(drop=True), train=True, train_augmentation=transformation)
    
    validation_dataset = MelData(df=train_df.iloc[train_idx].reset_index(drop=True), train=False, val=True, val_augmentation=transformation)
    
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=24, shuffle=True, num_workers=4)
    
    val_dataloader = DataLoader(dataset=validation_dataset, batch_size=24, shuffle=False, drop_last=True)
    
    model = timm.create_model(CFG['model'], pretrained=True, num_classes=1)
    model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    criterion.to(device)

    optim = torch.optim.AdamW(model.parameters(), lr= 0.001)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='max', patience=3, verbose=True, factor=0.2, min_lr=0.000001)

    train(model, optim, criterion, scheduler, train_dataloader, val_dataloader, fold)
        


loss is 92.785 and accuracy is 0.978
loss is 51.638 and auc is 0.877
